In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import string, nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VEADER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\VEADER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from numba import jit

In [20]:
df_fake = pd.read_csv('Fake.csv')
df_true = pd.read_csv('True.csv')

In [21]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [22]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [23]:
pattern = "^[A-Z/]+ \(Reuters\) - "
sum([1 for text in df_true["text"] if not re.match(pattern, text)])

5675

In [24]:
def clean(text: str) -> str:
    return re.sub(pattern, "", text)

In [25]:
clean(df_true["text"][0])

'The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support education, scientific researc

In [26]:
df_fake["text"] = df_fake["text"].apply(clean)
df_true["text"] = df_true["text"].apply(clean)
df_true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction ...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,Transgender people will be allowed for the fir...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,The special counsel investigation of links bet...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,Trump campaign adviser George Papadopoulos tol...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,President Donald Trump called on the U.S. Post...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,NATO allies on Tuesday welcomed President Dona...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LexisNexis, a provider of legal, regulatory an...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,In the shadow of disused Soviet-era factories ...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,Vatican Secretary of State Cardinal Pietro Par...,worldnews,"August 22, 2017"


In [27]:
df_fake = df_fake.drop(['date','subject'], axis=1)
df_true = df_true.drop(['date','subject'], axis=1)

In [28]:
df_true

,title,text
0,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction ...
1,U.S. military to accept transgender recruits o...,Transgender people will be allowed for the fir...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,The special counsel investigation of links bet...
3,FBI Russia probe helped by Australian diplomat...,Trump campaign adviser George Papadopoulos tol...
4,Trump wants Postal Service to charge 'much mor...,President Donald Trump called on the U.S. Post...
...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,NATO allies on Tuesday welcomed President Dona...
21413,LexisNexis withdrew two products from Chinese ...,"LexisNexis, a provider of legal, regulatory an..."
21414,Minsk cultural hub becomes haven from authorities,In the shadow of disused Soviet-era factories ...
21415,Vatican upbeat on possibility of Pope Francis ...,Vatican Secretary of State Cardinal Pietro Par...


In [30]:
df_fake["label"] = 0
df_true["label"] = 1
df_true

,title,text,label
0,"As U.S. budget fight looms, Republicans flip t...",The head of a conservative Republican faction ...,1
1,U.S. military to accept transgender recruits o...,Transgender people will be allowed for the fir...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,The special counsel investigation of links bet...,1
3,FBI Russia probe helped by Australian diplomat...,Trump campaign adviser George Papadopoulos tol...,1
4,Trump wants Postal Service to charge 'much mor...,President Donald Trump called on the U.S. Post...,1
...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,NATO allies on Tuesday welcomed President Dona...,1
21413,LexisNexis withdrew two products from Chinese ...,"LexisNexis, a provider of legal, regulatory an...",1
21414,Minsk cultural hub becomes haven from authorities,In the shadow of disused Soviet-era factories ...,1
21415,Vatican upbeat on possibility of Pope Francis ...,Vatican Secretary of State Cardinal Pietro Par...,1


In [31]:
df = pd.concat([df_fake, df_true])

In [32]:
df.to_csv("Fake new dataset.csv")

In [33]:
df = pd.read_csv("Fake new dataset.csv")
df.head()

,Unnamed: 0,title,text,label
0,0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0
1,1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0
2,2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0
3,3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0
4,4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0


In [34]:
df.isna().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [35]:
df= df.drop(['Unnamed: 0'], axis=1)
df.head()

,title,text,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0


In [36]:
stemmer = PorterStemmer()
@jit
def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

In [38]:
df['text']=df['text'].apply(stem_words)

In [39]:
lemmatizer = WordNetLemmatizer()
@jit
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [40]:
df['text']=df['text'].apply(lemmatize_words)

In [41]:
df.head()

,title,text,label
0,Donald Trump Sends Out Embarrassing New Year’...,donald trump just couldn t wish all american a...,0
1,Drunk Bragging Trump Staffer Started Russian ...,hous intellig committe chairman devin nune is ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,"on friday, it wa reveal that former milwauke s...",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"on christma day, donald trump announc that he ...",0
4,Pope Francis Just Called Out Donald Trump Dur...,pope franci use hi annual christma day messag ...,0


In [42]:
def text_process(review):
    nopunc = [char for char in review if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [43]:
df.dropna(inplace=True)

In [45]:
df['length'] = df['text'].apply(len)

In [46]:
df.groupby('label').describe()

length                                                                
         count         mean          std  min     25%     50%     75%      max
label                                                                         
0      23481.0  2333.002981  2298.043501  0.0  1316.0  1989.0  2780.0  46679.0
1      21417.0  2151.123126  1533.969259  0.0   819.0  2004.0  2923.0  27365.0

In [47]:
df.head()

,title,text,label,length
0,Donald Trump Sends Out Embarrassing New Year’...,donald trump just couldn t wish all american a...,0,2745
1,Drunk Bragging Trump Staffer Started Russian ...,hous intellig committe chairman devin nune is ...,0,1732
2,Sheriff David Clarke Becomes An Internet Joke...,"on friday, it wa reveal that former milwauke s...",0,3385
3,Trump Is So Obsessed He Even Has Obama’s Name...,"on christma day, donald trump announc that he ...",0,2557
4,Pope Francis Just Called Out Donald Trump Dur...,pope franci use hi annual christma day messag ...,0,2138


In [59]:
df['text'] = df['title'] + df['text']
df.drop(labels=['title'], axis=1, inplace=True)

In [60]:
df

,text,label,length
0,Donald Trump Sends Out Embarrassing New Year’...,0,2745
1,Drunk Bragging Trump Staffer Started Russian ...,0,1732
2,Sheriff David Clarke Becomes An Internet Joke...,0,3385
3,Trump Is So Obsessed He Even Has Obama’s Name...,0,2557
4,Pope Francis Just Called Out Donald Trump Dur...,0,2138
...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,1,2560
44894,LexisNexis withdrew two products from Chinese ...,1,706
44895,Minsk cultural hub becomes haven from authorit...,1,1755
44896,Vatican upbeat on possibility of Pope Francis ...,1,1102


In [62]:
review_train, review_test, label_train, label_test = train_test_split(df["text"],df['label'],test_size=0.15)

In [63]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',SVC())
])

In [64]:
pipeline.fit(review_train, label_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x0000027944EDAD30>)),
                ('tfidf', TfidfTransformer()), ('classifier', SVC())])

In [65]:
pred = pipeline.predict(review_test)

In [66]:
print(pred)


[1 0 0 ... 0 0 0]


In [67]:
from sklearn.metrics import accuracy_score
print('Accuracy Score:',accuracy_score(label_test,pred)*100)

Accuracy Score: 99.45063103192278


In [76]:
txt = input()
txt = stem_words(txt)
txt = lemmatize_words(txt)
txt = [txt]
if pipeline.predict(txt)== 1:
    print('True')
else:
    print('Fake')

Fake


In [77]:
from joblib import parallel,delayed
import joblib
joblib.dump(pipeline, 'model.pkl')

['model.pkl']